# Install

In [ ]:
pip install keras-self-attention

# Imports

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
from wordcloud import WordCloud

import re
import os
from os import walk
import io

import time
from timeit import default_timer as timer
from datetime import datetime

import string
import numpy as np
import pandas as pd
import scipy
from scipy import stats
from scipy.stats import entropy

from collections import Counter

import statistics
from statistics import mean, mode, stdev, median
import math

#from numpy import mean
from numpy import std

from sklearn import metrics
from sklearn.model_selection import train_test_split, StratifiedKFold, GridSearchCV
from sklearn.linear_model import SGDClassifier, RidgeClassifier
from sklearn.pipeline import make_pipeline, Pipeline
from sklearn.svm import SVC, LinearSVC
from sklearn import datasets, neighbors
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler, OneHotEncoder, PolynomialFeatures
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score
from sklearn.metrics import precision_recall_fscore_support
from sklearn.utils import shuffle
from sklearn.ensemble import RandomForestClassifier, ExtraTreesClassifier
from sklearn.naive_bayes import MultinomialNB, ComplementNB, GaussianNB
#import xgboost as xgb
from sklearn.neighbors import KNeighborsClassifier

from sklearn.neural_network import MLPClassifier
from sklearn.utils.extmath import softmax

from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer, TfidfTransformer

import random
from random import seed, randint, shuffle

import gensim
import nltk
from string import punctuation
from nltk import word_tokenize
from nltk.util import ngrams
from nltk.stem import WordNetLemmatizer, SnowballStemmer
from nltk.stem.porter import *
from nltk.stem import PorterStemmer
from nltk.corpus import stopwords

# nltk.download('stopwords')

#from tqdm import tqdm, trange
from tqdm.auto import tqdm

#from scikeras.wrappers import KerasClassifier

# BERT imports
import tensorflow as tf
#from tensorflow.keras.callbacks import ModelCheckpoint
from keras.layers import Dense, Flatten, Conv1D, GlobalMaxPool1D, Conv2D, GlobalMaxPool2D, Attention
from keras.layers import Dropout, Softmax, Embedding, Input, concatenate, SimpleRNN, LSTM, Masking
from keras import Model, layers, models
from keras.models import Sequential
from keras.initializers import Constant
#from keras_preprocessing.sequence import pad_sequences
#from keras_preprocessing.text import one_hot
from keras.optimizers.legacy import Adam
import keras
from keras import backend as K

from itertools import islice

from keras_self_attention import SeqSelfAttention

AUTOTUNE = tf.data.AUTOTUNE

In [ ]:
# Seed value
# Apparently you may use different seed values at each stage
seed_value= 42

# 1. Set the `PYTHONHASHSEED` environment variable at a fixed value
os.environ['PYTHONHASHSEED']=str(seed_value)

# 2. Set the `python` built-in pseudo-random generator at a fixed value
random.seed(seed_value)

# 3. Set the `numpy` pseudo-random generator at a fixed value
np.random.seed(seed_value)

# 4. Set the `tensorflow` pseudo-random generator at a fixed value
tf.random.set_seed(seed_value)
# for later versions:
# tf.compat.v1.set_random_seed(seed_value)

# 5. Configure a new global `tensorflow` session
session_conf = tf.compat.v1.ConfigProto(intra_op_parallelism_threads=1, inter_op_parallelism_threads=1)
sess = tf.compat.v1.Session(graph=tf.compat.v1.get_default_graph(), config=session_conf)
tf.compat.v1.keras.backend.set_session(sess)

# Drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [ ]:
path = '/content/drive/MyDrive/Areej/Attachment Emotions/'
#path = ''

# Data

In [ ]:
data = pd.read_csv(path+'Text/Data/attachment_emotions_text_v1.csv', header=0)

In [ ]:
data.rename({'ID': 'childID', 'Secure': 'insecure', 'Gender': 'gender', 'SchoolYear': 'schoolyear', 'Task': 'task',
                  'Path': 'path', 'Duration': 'duration', 'Start': 'start', 'End': 'end', 'Text': 'text'}, inplace=True, axis=1)

In [ ]:
# Switch secure and insecure
data.loc[data['insecure']==0, 'insecure'] = -1
data.loc[data['insecure']==1, 'insecure'] = 0
data.loc[data['insecure']==-1, 'insecure'] = 1

In [ ]:
data.drop(['gender', 'schoolyear', 'path', 'duration', 'start', 'end', 'text', 'time', 'segment'], axis=1, inplace=True)

In [ ]:
xDroppedColumns = ['childID', 'insecure', 'task']

In [ ]:
# keep_same = ['childID','task','insecure']
# dataB.columns = ['{}{}'.format(c, '' if c in keep_same else '_t')
#                for c in dataB.columns]

# dataA.columns = ['{}{}'.format(c, '' if c in keep_same else '_a')
#                for c in dataA.columns]

# Calculations

In [ ]:
def get_max_count(arr):#, prob0, prob1):
    # create a Counter object from the input array
    count_dict = Counter(arr)
    # find the element(s) with the maximum count
    max_count = max(count_dict.values())
    max_elements = [k for k, v in count_dict.items() if v == max_count]
    if len(max_elements) == 1:
        # return the element with the maximum count
        return max_elements[0]
    else:
        # Multiple similarly counted element, get the largest
        return max(max_elements)
        #return None
        #return 0 if prob0 > prob1 else 1

# Aggregate children's tasks MV
def getAggregatedTasksMVResults(resultsDF, isProb):
    aggTaskMVResDF = resultsDF.copy()

    # Grouping the data based on task
    aggTaskMVResDF = aggTaskMVResDF.groupby(['task','child_id'])

    def aggTaskMVData(data, isProb):
        df = {}
        df['child_id'] = data['child_id'].min()
        df['task'] = data['task'].min()
        df['truth'] = int(data['truth'].min())
        df['prediction'] = get_max_count(data['prediction'].to_list())#, round(data['prob0'].mean(),2), round(data['prob1'].mean(),2))

        pred1 = int(df['prediction'])
        pred2 = 0 if pred1 == 1 else 1

        if isProb: # prob is calculated
            df['prob'+str(pred1)] = round(data['prob'+str(pred1)].mean(),2)
            df['prob'+str(pred2)] = round(data['prob'+str(pred2)].mean(),2)
        else:
            pred1Len = len(data[data.prediction==df['prediction']])
            pred1 = int(df['prediction'])
            pred2 = 0 if pred1 == 1 else 1
            total = data['prediction'].count()

            df['prob'+str(pred1)] = round(pred1Len/total,2)
            df['prob'+str(pred2)] = 1-df['prob'+str(pred1)]

        return pd.Series(df, index=['child_id', 'task', 'truth', 'prediction', 'prob0', 'prob1'])

    # Aggregating the data
    aggTaskMVResDF = aggTaskMVResDF.apply(aggTaskMVData, isProb)
    # Drop the multiindex
    aggTaskMVResDF.reset_index(drop=True, inplace=True)

    return aggTaskMVResDF

# Aggregate children's tasks WV
def getAggregatedTasksWVResults(resultsDF, isProb):
    aggTaskWVResDF = resultsDF.copy()

    # Grouping the data based on task
    aggTaskWVResDF = aggTaskWVResDF.groupby(['task','child_id'])

    def aggTaskWVData(data):
        df = {}
        df['child_id'] = data['child_id'].min()
        df['task'] = data['task'].min()
        df['truth'] = data['truth'].min()

        if len(data['prediction'].unique()) > 1:
            pred1 = int(data['prediction'].sort_values().unique()[0])
            pred2 = int(data['prediction'].sort_values().unique()[1])

            # Consider only when prediction is correct
            #prob1 = data[data['prediction']==pred1]['prob'+str(pred1)].mean()
            #prob2 = data[data['prediction']==pred2]['prob'+str(pred2)].mean()

            # Add up all probabilities regardless of prediction
            prob1 = round(data['prob'+str(pred1)].mean(),2)
            prob2 = round(data['prob'+str(pred2)].mean(),2)

            df['prediction'] = pred1 if prob1 > prob2 else pred2 # if prob equals, favore the sick prediction
            df['prob'+str(pred1)] = prob1
            df['prob'+str(pred2)] = prob2
        else:
            df['prediction'] = data['prediction'].min()

            pred1 = int(df['prediction'])
            pred2 = 0 if pred1 == 1 else 1

            df['prob'+str(pred1)] = round(data['prob'+str(pred1)].mean(),2)
            df['prob'+str(pred2)] = round(data['prob'+str(pred2)].mean(),2)

        return pd.Series(df, index=['child_id', 'task', 'truth', 'prediction', 'prob0', 'prob1'])

    # Aggregating the data
    aggTaskWVResDF = aggTaskWVResDF.apply(aggTaskWVData)
    # Drop the multiindex
    aggTaskWVResDF.reset_index(drop=True, inplace=True)

    return aggTaskWVResDF

# Aggregate children's tasks Ratio
def getAggregatedTasksRatioResults(resultsDF, ratio, isProb):
    aggTaskRatioResDF = resultsDF.copy()

    # Grouping the data based on task
    aggTaskRatioResDF = aggTaskRatioResDF.groupby(['task','child_id'])

    def aggTaskRatioData(data, isProb):
        df = {}
        df['child_id'] = data['child_id'].min()
        df['task'] = data['task'].min()
        df['truth'] = data['truth'].min()

        total = data['prediction'].count()

        if len(data['prediction'].unique()) > 1:
            pred1 = data['prediction'].sort_values().unique()[0]
            count1 = data[data['prediction']==pred1].prediction.count()
            prob1 = round(count1/total*100,2)

            pred2 = data['prediction'].sort_values().unique()[1]
            count2 = data[data['prediction']==pred2].prediction.count()
            prob2 = round(count2/total*100,2)

            df['prediction'] = pred2 if prob2 >= ratio else pred1 # math.ceil(data['total']*ratio)
        else:
            df['prediction'] = max(set(data['prediction']), key = data['prediction'].to_list().count)

        pred = int(df['prediction'])
        pred = 0 if pred == -1 else pred

        if isProb:
            df['prob'] =  round(data['prob'+str(pred)].min()*100,2)
        else :
            pred = len(data[data.prediction==df['prediction']])
            total = data['prediction'].count()
            df['prob'] = round(pred/total*100,2)

        df['total'] = data['prediction'].count()

        return pd.Series(df, index=['child_id', 'task', 'truth', 'prediction', 'prob', 'total'])

    # Aggregating the data
    aggTaskRatioResDF = aggTaskRatioResDF.apply(aggTaskRatioData, isProb)
    # Drop the multiindex
    aggTaskRatioResDF.reset_index(drop=True, inplace=True)

    return aggTaskRatioResDF

In [ ]:
# Aggregate children MV
def getAggregatedChildrenResultsMV(resultsDF):
    aggChildMajVResultsDF = resultsDF.copy()

    # Grouping the data based on children
    aggChildMajVResultsDF = aggChildMajVResultsDF.groupby(['child_id'])

    def aggChildMajVData(data):
        df = {}
        df['child_id'] = data['child_id'].min()
        df['truth'] = data['truth'].min()
        df['prediction'] = get_max_count(data['prediction'].to_list())#, round(data['prob0'].mean(),2), round(data['prob1'].mean(),2))
        pred1, pred2 = 0, 1
        df['prob'+str(pred1)] = round(data['prob'+str(pred1)].mean(),2)
        df['prob'+str(pred2)] = round(data['prob'+str(pred2)].mean(),2)

        return pd.Series(df, index=['child_id', 'truth', 'prediction', 'prob0', 'prob1'])

    # Aggregating the data
    aggChildMajVResultsDF = aggChildMajVResultsDF.apply(aggChildMajVData)
    # Drop the multiindex
    aggChildMajVResultsDF.reset_index(drop=True, inplace=True)

    return aggChildMajVResultsDF

# Aggregate children WV
def getAggregatedChildrenResultsWV(resultsDF):
    # Grouping the data based on children
    aggChildWVResultsDF = resultsDF.copy().groupby(['child_id'])

    def aggChildWVData(data):
        df = {}
        df['child_id'] = data['child_id'].min()
        df['truth'] = data['truth'].min()

        if len(data['prediction'].unique()) > 1:
            pred1 = int(data['prediction'].sort_values().unique()[0])
            pred2 = int(data['prediction'].sort_values().unique()[1])

            # Consider only when prediction is correct
            #prob1 = data[data['prediction']==pred1]['prob'+str(pred1)].mean()
            #prob2 = data[data['prediction']==pred2]['prob'+str(pred2)].mean()

            # Add up all probabilities regardless of prediction
            prob1 = round(data['prob'+str(pred1)].mean(),2)
            prob2 = round(data['prob'+str(pred2)].mean(),2)

            df['prediction'] = pred1 if prob1 > prob2 else pred2 # if prob equals, favore the sick prediction
            df['prob'+str(pred1)] = prob1
            df['prob'+str(pred2)] = prob2
        else:
            df['prediction'] = data['prediction'].min()

            pred1 = int(df['prediction'])
            pred2 = 0 if pred1 == 1 else 1

            df['prob'+str(pred1)] = round(data['prob'+str(pred1)].mean(),2)
            df['prob'+str(pred2)] = round(data['prob'+str(pred2)].mean(),2)

        return pd.Series(df, index=['child_id', 'truth', 'prediction', 'prob0', 'prob1'])

    # Aggregating the data # Drop the multiindex
    aggChildWVResultsDF = aggChildWVResultsDF.apply(aggChildWVData).reset_index(drop=True)

    return aggChildWVResultsDF

# Aggregate children ratio
def getAggregatedChildrenResultsRatio(resultsDF, ratio):
    aggChildRatioResultsDF = resultsDF.copy()

    # Grouping the data based on children
    aggChildRatioResultsDF = aggChildRatioResultsDF.groupby(['child_id'])

    def aggChildRatioData(data):
        df = {}
        df['child_id'] = data['child_id'].min()
        df['truth'] = data['truth'].min()

        total = data['prediction'].count()

        if len(data['prediction'].unique()) > 1:
            pred1 = data['prediction'].sort_values().unique()[0]
            count1 = data[data['prediction']==pred1].prediction.count()
            prob1 = round(count1/total*100,2)

            pred2 = data['prediction'].sort_values().unique()[1]
            count2 = data[data['prediction']==pred2].prediction.count()
            prob2 = round(count2/total*100,2)

            df['prediction'] = pred2 if prob2 >= ratio else pred1 # math.ceil(data['total']*ratio)
        else:
            df['prediction'] = max(set(data['prediction']), key = data['prediction'].to_list().count)

        return pd.Series(df, index=['child_id', 'truth', 'prediction'])
    # Aggregating the data
    aggChildRatioResultsDF = aggChildRatioResultsDF.apply(aggChildRatioData)
    # Drop the multiindex
    aggChildRatioResultsDF.reset_index(drop=True, inplace=True)

    return aggChildRatioResultsDF

# Aggregate children from aggregated tasks (ratio)
def getAggregatedTaskChildrenResultsRatioV(aggTaskMVResDF, ratio):
    aggChildTaskMajRatioVResultsDF = aggTaskMVResDF.copy()

    # Grouping the data based on children
    aggChildTaskMajRatioVResultsDF = aggChildTaskMajRatioVResultsDF.groupby(['child_id'])

    def aggChildRatioVData(data):
        df = {}
        df['child_id'] = data['child_id'].min()
        df['truth'] = data['truth'].min()

        total = data['prediction'].count()

        if len(data['prediction'].unique()) > 1:
            pred1 = data['prediction'].sort_values().unique()[0]
            count1 = data[data['prediction']==pred1].prediction.count()
            prob1 = round(count1/total*100,2)

            pred2 = data['prediction'].sort_values().unique()[1]
            count2 = data[data['prediction']==pred2].prediction.count()
            prob2 = round(count2/total*100,2)

            df['prediction'] = pred2 if prob2 >= ratio else pred1 # math.ceil(data['total']*ratio)
        else:
            df['prediction'] = max(set(data['prediction']), key = data['prediction'].to_list().count)

        return pd.Series(df, index=['child_id', 'truth', 'prediction'])
    # Aggregating the data
    aggChildTaskMajRatioVResultsDF = aggChildTaskMajRatioVResultsDF.apply(aggChildRatioVData)
    # Drop the multiindex
    aggChildTaskMajRatioVResultsDF.reset_index(drop=True, inplace=True)

    return aggChildTaskMajRatioVResultsDF

In [ ]:
def calSResults(resultsDF, isProb=True):
    repResults = pd.DataFrame(columns=['model', 'task','acc','prec','rec','f1'])

#     resultsDF.truth = resultsDF.truth.astype(int)
#     resultsDF.prediction = resultsDF.prediction.astype(int)

    aggTaskMVResDF = getAggregatedTasksMVResults(resultsDF, isProb)

    # Story level
    for task in aggTaskMVResDF.task.unique():
        repResults.loc[len(repResults)] = ['STMV', task,
            round(accuracy_score(aggTaskMVResDF[aggTaskMVResDF.task==task].truth, aggTaskMVResDF[aggTaskMVResDF.task==task].prediction)*100,1),
            round(metrics.precision_score(aggTaskMVResDF[aggTaskMVResDF.task==task].truth, aggTaskMVResDF[aggTaskMVResDF.task==task].prediction, average='binary', pos_label=1)*100,1),
            round(metrics.recall_score(aggTaskMVResDF[aggTaskMVResDF.task==task].truth, aggTaskMVResDF[aggTaskMVResDF.task==task].prediction, average='binary', pos_label=1)*100,1),
            round(metrics.f1_score(aggTaskMVResDF[aggTaskMVResDF.task==task].truth, aggTaskMVResDF[aggTaskMVResDF.task==task].prediction, average='binary', pos_label=1)*100,1)]

    # Story agg -> child agg

    # Child MV
    aggChildMajVResultsDF = getAggregatedChildrenResultsMV(aggTaskMVResDF)
    repResults.loc[len(repResults)] = ['STMVCMV', 0,
          round(accuracy_score(aggChildMajVResultsDF.truth, aggChildMajVResultsDF.prediction)*100,1),
          round(metrics.precision_score(aggChildMajVResultsDF.truth, aggChildMajVResultsDF.prediction, average='binary', pos_label=1)*100,1),
          round(metrics.recall_score(aggChildMajVResultsDF.truth, aggChildMajVResultsDF.prediction, average='binary', pos_label=1)*100,1),
          round(metrics.f1_score(aggChildMajVResultsDF.truth, aggChildMajVResultsDF.prediction, average='binary', pos_label=1)*100,1)]

    # Child WV
    aggChildTaskMajWResultsDF = getAggregatedChildrenResultsWV(aggTaskMVResDF)
    repResults.loc[len(repResults)] = ['STMVCWV', 0,
          round(accuracy_score(aggChildTaskMajWResultsDF.truth, aggChildTaskMajWResultsDF.prediction)*100,1),
          round(metrics.precision_score(aggChildTaskMajWResultsDF.truth, aggChildTaskMajWResultsDF.prediction, average='binary', pos_label=1)*100,1),
          round(metrics.recall_score(aggChildTaskMajWResultsDF.truth, aggChildTaskMajWResultsDF.prediction, average='binary', pos_label=1)*100,1),
          round(metrics.f1_score(aggChildTaskMajWResultsDF.truth, aggChildTaskMajWResultsDF.prediction, average='binary', pos_label=1)*100,1)]

    return repResults


# Model

In [ ]:
def keras_f1_score(y_true, y_pred): #taken from old keras source code
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    recall = true_positives / (possible_positives + K.epsilon())
    f1_val = 2*(precision*recall)/(precision+recall+K.epsilon())
    return f1_val

In [ ]:
def attachment_model(sequence_length, num_columns, n_classes, lstm_units, att_units, att_type, att_window, att_hist, att_act, dense_units):
    # lstm_units=64, att_window=3, dense_units=30
    # Model
    tf.keras.backend.clear_session()

    # Opt
    opt = Adam(learning_rate=0.001, beta_1=0.9, beta_2=0.999, decay=0.01)

    model = Sequential()
    model.add(Masking(mask_value=0., input_shape=(sequence_length, num_columns)))
    model.add(LSTM(lstm_units, seed=seed_value, return_sequences=True)) # units means the copies of the rnn cell, no relation to embedding size
    model.add(SeqSelfAttention(units=att_units, # TODO
            attention_type=att_type, # TODO: ATTENTION_TYPE_MUL ATTENTION_TYPE_ADD **
            kernel_initializer=keras.initializers.GlorotNormal(seed=seed_value),
            kernel_regularizer=keras.regularizers.l2(1e-4),
            bias_regularizer=keras.regularizers.l1(1e-4),
            attention_regularizer_weight=1e-4,
            attention_width=att_window, # TODO
            history_only=att_hist, # TODO **
            attention_activation=att_act, # TODO **
            name='Attention'))
    model.add(Flatten())
    model.add(Dense(dense_units, activation='relu'))
    model.add(Dropout(0.2))
    model.add(Dense(n_classes, activation='sigmoid')) #'linear'

    model.compile(optimizer=opt, #'categorical_crossentropy'
        loss='binary_crossentropy', #tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
        metrics=[keras_f1_score]) # 'accuracy'
    #model.summary()

    return model

## Data

In [ ]:
class CustomCSVLogger(tf.keras.callbacks.CSVLogger):
    def __init__(self, cusParams, filename, separator=',', append=False):
        super().__init__(filename, separator, append)

        self.cusParams = cusParams

    def on_epoch_end(self, epoch, logs=None):
        # You can access the logs dictionary which contains metrics like loss and accuracy
        # Add your custom information to logs dictionary
        for k, v in self.cusParams.items():
            logs[k] = v

        # Call the parent class's on_epoch_end method to log the standard information
        super().on_epoch_end(epoch, logs)

In [ ]:
import os
import numpy as np
import tensorflow as tf

class ReadEmoWeightsDataset:
    def __init__(self, data_folder, df, data_indices, batch_size=32):
        self.data_folder = data_folder
        self.file_list = os.listdir(data_folder)
        self.dataDF = df

        max_seq_length = 0
        for file in self.file_list:
            seg = int(file.split('_')[1].split('.')[0])
            max_seq_length = seg if seg > max_seq_length else max_seq_length

        self.max_seq_length = max_seq_length
        self.children_ids = data_indices
        self.batch_size = batch_size

    # Custom key function to extract the numeric part before '_'
    def key_func(self, s):
        # if ind == 0:
        #     return int(s.split('_')[ind])
        # elif ind == 1:
        return int(s.split('_')[1].split('.')[0])

    def collect_data(self, idx):
        child_id = list(self.children_ids)[idx]

        # Find all files belonging to the same person
        children_files = [file_name for file_name in self.file_list if file_name.startswith(str(child_id)+'_')]
        sorted_children_files = sorted(children_files, key=lambda x: self.key_func(x))

        # Read data from files and combine into array of arrays
        array = []
        for file_name in sorted_children_files:
            file_path = os.path.join(self.data_folder, file_name)
            child_data = np.load(file_path)['arr_0'][0]
            array.append(child_data)
        array = np.array(array)

        # Calculate the number of rows to pad
        rows_to_pad = max(0, self.max_seq_length - array.shape[0])

        # Pad the array along the first dimension
        padding = np.zeros((rows_to_pad, *array.shape[1:]), dtype=array.dtype)
        padded_array = np.concatenate((array, padding), axis=0)

        combined_array = np.array(padded_array)

        # Get label from the data dataframe
        label = int(self.dataDF[self.dataDF.childID==int(child_id)].insecure.values[0])

        return combined_array, label


    def __len__(self):
        return len(self.children_ids)

    def __iter__(self):
        self.index = 0
        return self

    def __next__(self):
        if self.index >= len(self):
            raise StopIteration

        batch_data = []
        batch_labels = []

        for idx in range(self.index, min(self.index + self.batch_size, len(self))):
            combined_array, label = self.collect_data(idx)

            batch_data.append(combined_array)
            batch_labels.append(label)

        self.index += self.batch_size

        return np.array(batch_data), np.array(batch_labels)

    def __getitem__(self, idx):
        if idx >= len(self):
            raise IndexError("Index out of range")

        batch_data = []
        batch_labels = []

        combined_array, label = self.collect_data(idx)

        batch_data.append(combined_array)
        batch_labels.append(label)

        return np.array(batch_data), np.array(batch_labels)


# Fine-Tune Model

## Parameters

In [ ]:
class cConfigs:
    def __init__(self, lstm_units, dense_units, att_window, att_units, att_type, att_hist, att_act,
                 batch_size=4, epochs=20, dropout=0.2, lr=0.001):

        self.batch_size = batch_size
        self.epochs = epochs

        self.lstm_units = lstm_units

        self.dense_units = dense_units # 64
        self.dropout = dropout # , 0.4
        self.lr = lr # , 0.0001

        self.att_units = att_units # 32
        self.att_type = att_type #, SeqSelfAttention.ATTENTION_TYPE_ADD]
        self.att_window = att_window
        self.att_hist =  att_hist #, False]
        self.att_act = att_act # None,

        self.params = {'batch_size': batch_size, 'epochs': epochs, 'lstm_units':lstm_units,
                       #'conv_filters': conv_filters, 'conv_kernel_size': conv_kernel_size, 'strides': strides,
                       'dense_units': dense_units, 'dropout': dropout, 'lr': lr,
                       'att_units': att_units, 'att_type': att_type, 'att_window': att_window,
                       'att_hist': att_hist, 'att_act': att_act}

    def set_param(self, newParams):
        for k, v in newParams.items():
            self.params[k] = v

    def get_params(self):
        return self.params

    def get_tuned_params(self):
        tuned = {}

        for k, v in self.params.items():
            if type(v) == list:
                tuned[k] = v
        return tuned

    def get_nontuned_params(self):
        tuned = {}

        for k, v in self.params.items():
            if type(v) != list:
                tuned[k] = v
        return tuned

In [ ]:
# Create All combinations

def getCombinations(obj, n_nested_levels):
    curLevelArr = []

    # If this is the last nested level, return the elemnts as individual arrays
    if n_nested_levels == 0:
        for item in obj[0]:
            curLevelArr.append([item])

        return curLevelArr

    else:
        # Call the function again
        subLevelArr = getCombinations(obj[1:], n_nested_levels-1)

        # Add the current level items to the existing sublists
        for subList in subLevelArr:
            for element in obj[0]: # current level of the nested structure
                curLevelArr.append([element]+subList)

    return curLevelArr

In [ ]:
type1configs = cConfigs(lstm_units=[16, 32, 64], dense_units=[8, 16, 32], att_window=[2, 4, 8],
                        att_units=[None], att_type=['multiplicative'],
                        att_hist=[True, False], att_act=[None, 'Softmax'])

type2configs = cConfigs(lstm_units=[16, 32, 64], dense_units=[8, 16, 32], att_window=[2, 4, 8],
                   att_units=[8, 16, 32], att_type=['additive'],
                   att_hist=[True, False], att_act=[None, 'Softmax'])

selParams1 = type1configs.get_tuned_params()
selParams2 = type2configs.get_tuned_params()

In [ ]:
possibleConfigs1 = []
params2tune1 = list(selParams1.values())
possibleConfigs1 = getCombinations(params2tune1, len(params2tune1)-1)

possibleConfigs2 = []
params2tune2 = list(selParams2.values())
possibleConfigs2 = getCombinations(params2tune2, len(params2tune2)-1)

In [ ]:
len(possibleConfigs1), len(possibleConfigs2), len(possibleConfigs1) + len(possibleConfigs2)

(108, 324, 432)

In [ ]:
allPossibleConfigs = possibleConfigs1 + possibleConfigs2

selectedConfigs = random.sample(allPossibleConfigs, 20) #### 1/3rd of possiple combinations

In [ ]:
for i, cfg in enumerate(selectedConfigs):
    print(i+1, cfg)

1 [16, 16, 8, 'additive', 8, True, 'Softmax']
2 [16, 16, None, 'multiplicative', 2, True, 'Softmax']
3 [16, 16, None, 'multiplicative', 4, True, None]
4 [32, 8, 8, 'additive', 4, False, 'Softmax']
5 [64, 16, 8, 'additive', 4, True, None]
6 [64, 32, 16, 'additive', 2, True, None]
7 [16, 32, 8, 'additive', 2, True, None]
8 [64, 32, None, 'multiplicative', 4, True, 'Softmax']
9 [64, 32, 32, 'additive', 2, False, 'Softmax']
10 [32, 32, None, 'multiplicative', 8, False, None]
11 [32, 16, 32, 'additive', 8, True, 'Softmax']
12 [16, 8, 16, 'additive', 2, True, 'Softmax']
13 [64, 32, None, 'multiplicative', 4, False, None]
14 [64, 16, 8, 'additive', 4, True, 'Softmax']
15 [16, 8, 8, 'additive', 4, False, None]
16 [32, 32, None, 'multiplicative', 4, True, None]
17 [16, 32, None, 'multiplicative', 4, True, None]
18 [64, 8, None, 'multiplicative', 8, False, None]
19 [16, 16, 8, 'additive', 2, True, None]
20 [64, 8, 16, 'additive', 2, True, None]


## Code

In [ ]:
# Nested CV
n_outer_folds = 4
n_inner_folds = 3
random_state = 42

n_classes = 1
num_columns = 768

resultColumns = ['truth', 'prediction', 'prob0', 'prob1', 'child_id', 'task', 'ofold', 'ifold', 'run', 'source', 'elapsedtime', 'experiment']

save_path = path+'Text/Finetuned/' # maxpooling

for experiment, choices in enumerate(tqdm(selectedConfigs, desc=" exp", position=0, leave=True)):
    experiment += 1

    expParams = type1configs.get_nontuned_params()

    for k, v in zip(list(selParams1.keys()),choices):
        expParams[k] = v

    model_path = save_path+str(experiment)+'/'
    # Create folders
    if str(experiment) not in os.listdir(save_path): os.makedirs(model_path)

    for run in tqdm(range(1, 4), desc=" runs", position=1, leave=False): # 3 runs # TODO
        start_time = time.time()

        cp_path = model_path+str(run)+'/'
        # Create folders
        if str(run) not in os.listdir(model_path): os.makedirs(cp_path)

        # Go through tasks
        for task in tqdm(data.task.unique(), desc=" tasks", position=2, leave=False): # TODO
            start_time = time.time()

            resultsTr = []
            resultsVal = []
            resultsTs = []

            # Get task data
            workingData = data[data.task == task].sort_values('childID').reset_index(drop = True)

            sequence_length = workingData.groupby(['childID']).count().task.max() # use any column

            childIDs = workingData.childID.unique()
            childLabels = workingData.groupby('childID').first().insecure.values

            data_folder = path+'Text/Data/Weights/{}/'.format(task)

            # Define the outer kfold
            kfold_test = StratifiedKFold(n_outer_folds, shuffle=True, random_state=random_state)

            for ofold, (train_indices, test_indices) in enumerate(tqdm(kfold_test.split(childIDs, childLabels), total=n_outer_folds, desc=" ofolds", position=3, leave=False)):
                # Get the train data
                innerChildIDs = workingData[workingData.childID.isin(childIDs[train_indices])].childID.unique()
                innerChildLabels = workingData[workingData.childID.isin(childIDs[train_indices])].groupby('childID').first().insecure.values

                # Define the inner fold
                kfold_val = StratifiedKFold(n_inner_folds, shuffle=True, random_state=random_state)

                for ifold, (inner_train_indices, val_indices) in enumerate(tqdm(kfold_val.split(innerChildIDs, innerChildLabels), total=n_inner_folds, desc=" ifolds", position=4, leave=False)):
                    # Inner Train set
                    dataset_train = ReadEmoWeightsDataset(data_folder, workingData, innerChildIDs[inner_train_indices], batch_size=len(innerChildIDs[inner_train_indices]))
                    tf_dataset_train = tf.data.Dataset.from_generator(dataset_train.__iter__, (tf.float32, tf.int32), ((None, None, None), (None,)))
                    x_train, y_train = next(iter(tf_dataset_train))

                    # Val set
                    dataset_val = ReadEmoWeightsDataset(data_folder, workingData, innerChildIDs[val_indices], batch_size=len(innerChildIDs[val_indices]))
                    tf_dataset_val = tf.data.Dataset.from_generator(dataset_val.__iter__, (tf.float32, tf.int32), ((None, None, None), (None,)))
                    x_val, y_val = next(iter(tf_dataset_val))

                    # Model
                    model = attachment_model(sequence_length, num_columns, n_classes,
                                             lstm_units=expParams['lstm_units'],
                                             att_window=expParams['att_window'], dense_units=expParams['dense_units'], att_units=expParams['att_units'],
                                             att_type=expParams['att_type'], att_hist=expParams['att_hist'], att_act=expParams['att_act'])


                    # A callback that saves the model's weights
                    cp_callback = tf.keras.callbacks.ModelCheckpoint(filepath=cp_path, save_weights_only=True, verbose=0,
                                                                    monitor="val_keras_f1_score", mode="max", save_best_only=True)

                    expParams['task'], expParams['ofold'], expParams['ifold'] = task, ofold, ifold


                    # Logging performacen callback
                    history_callback = CustomCSVLogger(expParams, cp_path+'log.csv', separator=",", append=True)

                    # Early stopping callback
                    estopping_callback = keras.callbacks.EarlyStopping(monitor="val_keras_f1_score", patience=3, min_delta=0.001, mode="max", restore_best_weights=False)

                    history = model.fit(x_train, y_train, validation_data=(x_val, y_val), verbose=0,
                                        epochs=expParams['epochs'], batch_size=expParams['batch_size'],
                                        callbacks=[history_callback, estopping_callback, cp_callback])
                    # Retrieve best model
                    # The model weights (that are considered the best) can be loaded as -
                    model.load_weights(cp_path)

                    # Evaluation
                    pred_inner_train = model(x_train)
                    pred_val = model(x_val)

                    end_time = time.time()

                    # Save results
                    resultsTr.append([y_train, pred_inner_train, innerChildIDs[inner_train_indices], task, ofold, ifold, run, end_time - start_time, experiment])
                    resultsVal.append([y_val, pred_val, innerChildIDs[val_indices], task, ofold, ifold, run, end_time - start_time, experiment])


            resultsDF = pd.DataFrame(columns=resultColumns)

            for row in resultsTr:
                resultTempDF = pd.DataFrame()
                resultTempDF[resultColumns[0]] = row[0]#.values
                resultTempDF[resultColumns[1]] = [0 if prob1.numpy().tolist()[0] < 0.5 else 1 for prob1 in row[1]]
                resultTempDF[resultColumns[2]] = [1 - prob1.numpy().tolist()[0] for prob1 in row[1]]
                resultTempDF[resultColumns[3]] = [prob1.numpy().tolist()[0] for prob1 in row[1]]
                resultTempDF[resultColumns[4]] = row[2]
                resultTempDF[resultColumns[5]] = row[3]
                resultTempDF[resultColumns[6]] = row[4]
                resultTempDF[resultColumns[7]] = row[5]
                resultTempDF[resultColumns[8]] = row[6]
                resultTempDF[resultColumns[9]] = 'train'
                resultTempDF[resultColumns[10]] = row[7]
                resultTempDF[resultColumns[11]] = row[8]
                resultsDF = pd.concat([resultsDF, resultTempDF], axis=0)

            for row in resultsVal:
                resultTempDF = pd.DataFrame()
                resultTempDF[resultColumns[0]] = row[0]#.values
                resultTempDF[resultColumns[1]] = [0 if prob1.numpy().tolist()[0] < 0.5 else 1 for prob1 in row[1]]
                resultTempDF[resultColumns[2]] = [1 - prob1.numpy().tolist()[0] for prob1 in row[1]]
                resultTempDF[resultColumns[3]] = [prob1.numpy().tolist()[0] for prob1 in row[1]]
                resultTempDF[resultColumns[4]] = row[2]
                resultTempDF[resultColumns[5]] = row[3]
                resultTempDF[resultColumns[6]] = row[4]
                resultTempDF[resultColumns[7]] = row[5]
                resultTempDF[resultColumns[8]] = row[6]
                resultTempDF[resultColumns[9]] = 'val'
                resultTempDF[resultColumns[10]] = row[7]
                resultTempDF[resultColumns[11]] = row[8]
                resultsDF = pd.concat([resultsDF, resultTempDF], axis=0)

            resultsDF.reset_index(inplace=True, drop=True)
            #resultsDF.to_csv('ResultsNNLIWC/speech_results_liwc_cnn_v5_e20_runs'+str(run)+'_v13.csv',index=False)

            with open(save_path+'Results/speech_emo_e_'+str(experiment)+'.csv', 'a') as f1:
                resultsDF.to_csv(f1, mode='a', index=False, header=f1.tell()==0)



 exp:   0%|          | 0/1 [00:00<?, ?it/s]

 runs:   0%|          | 0/3 [00:00<?, ?it/s]

 tasks:   0%|          | 0/5 [00:00<?, ?it/s]

 ofolds:   0%|          | 0/4 [00:00<?, ?it/s]

 ifolds:   0%|          | 0/3 [00:00<?, ?it/s]

...

 ofolds:   0%|          | 0/4 [00:00<?, ?it/s]

 ifolds:   0%|          | 0/3 [00:00<?, ?it/s]

 ifolds:   0%|          | 0/3 [00:00<?, ?it/s]

 ifolds:   0%|          | 0/3 [00:00<?, ?it/s]

 ifolds:   0%|          | 0/3 [00:00<?, ?it/s]

# Final Model

In [ ]:
# Nested CV
n_outer_folds = 4
n_inner_folds = 3
random_state = 42

n_classes = 1
num_columns = 768

resultColumns = ['truth', 'prediction', 'prob0', 'prob1', 'child_id', 'task', 'ofold', 'run', 'source', 'elapsedtime', 'experiment']

save_path = path+'Text/Final/'
#save_res_path = path+'Text/'

for experiment, choices in enumerate(tqdm(selectedConfigs[4:5], desc=" exp", position=0, leave=True)): # TODO
    experiment += 5 # TODO

    expParams = type1configs.get_nontuned_params()

    for k, v in zip(list(selParams1.keys()),choices):
        expParams[k] = v

    model_path = save_path+str(experiment)+'/'
    # Create folders
    if str(experiment) not in os.listdir(save_path): os.makedirs(model_path)

    for run in tqdm(range(4, 11), desc=" runs", position=1, leave=False): # 3 runs # TODO
        start_time = time.time()

        cp_path = model_path+str(run)+'/'
        # Create folders
        if str(run) not in os.listdir(model_path): os.makedirs(cp_path)

        # Go through tasks
        for task in tqdm(data.task.unique(), desc=" tasks", position=2, leave=False): # TODO
            start_time = time.time()

            resultsTr = []
            resultsVal = []
            resultsTs = []

            # Get task data
            workingData = data[data.task == task].sort_values('childID').reset_index(drop = True)

            sequence_length = workingData.groupby(['childID']).count().task.max() # use any column

            childIDs = workingData.childID.unique()
            childLabels = workingData.groupby('childID').first().insecure.values

            data_folder = path+'Text/Data/Weights/{}/'.format(task)

            # Define the outer kfold
            kfold_test = StratifiedKFold(n_outer_folds, shuffle=True, random_state=random_state)

            for ofold, (train_indices, test_indices) in enumerate(tqdm(kfold_test.split(childIDs, childLabels), total=n_outer_folds, desc=" ofolds", position=3, leave=False)):
                # Train set
                dataset_train = ReadEmoWeightsDataset(data_folder, workingData, childIDs[train_indices], batch_size=len(childIDs[train_indices]))
                tf_dataset_train = tf.data.Dataset.from_generator(dataset_train.__iter__, (tf.float32, tf.int32), ((None, None, None), (None,)))
                x_train, y_train = next(iter(tf_dataset_train))

                dataset_test = ReadEmoWeightsDataset(data_folder, workingData, childIDs[test_indices], batch_size=len(childIDs[test_indices]))
                tf_dataset_test = tf.data.Dataset.from_generator(dataset_test.__iter__, (tf.float32, tf.int32), ((None, None, None), (None,)))
                x_test, y_test = next(iter(tf_dataset_test))

                # Model
                model = attachment_model(sequence_length, num_columns, n_classes,
                                        #conv_filters=expParams['conv_filters'], conv_kernel_size=expParams['conv_kernel_size'], strides=expParams['strides'],
                                        lstm_units=expParams['lstm_units'],
                                        att_window=expParams['att_window'], dense_units=expParams['dense_units'], att_units=expParams['att_units'],
                                        att_type=expParams['att_type'], att_hist=expParams['att_hist'], att_act=expParams['att_act'])

                # A callback that saves the model's weights
                cp_callback = tf.keras.callbacks.ModelCheckpoint(filepath=cp_path, save_weights_only=True, verbose=0,
                                                                monitor="keras_f1_score", mode="max", save_best_only=True)

                expParams['task'], expParams['ofold'] = task, ofold


                # Logging performacen callback
                history_callback = CustomCSVLogger(expParams, cp_path+'log.csv', separator=",", append=True)

                # Early stopping callback
                estopping_callback = keras.callbacks.EarlyStopping(monitor="keras_f1_score", patience=3, min_delta=0.001, mode="max", restore_best_weights=False)

                history = model.fit(x_train, y_train, verbose=0,
                                   epochs=expParams['epochs'], batch_size=expParams['batch_size'],
                                   callbacks=[history_callback, estopping_callback, cp_callback])
                # Retrieve best model
                model.load_weights(cp_path)

                # Evaluation
                pred_train = model(x_train)
                pred_test = model(x_test)

                end_time = time.time()

                # Save results
                resultsTr.append([y_train, pred_train, childIDs[train_indices], task, ofold, run, end_time - start_time, experiment])
                resultsTs.append([y_test, pred_test, childIDs[test_indices], task, ofold, run, end_time - start_time, experiment])


            resultsDF = pd.DataFrame(columns=resultColumns)

            for row in resultsTr:
                resultTempDF = pd.DataFrame()
                resultTempDF[resultColumns[0]] = row[0]#.values
                resultTempDF[resultColumns[1]] = [0 if prob1.numpy().tolist()[0] < 0.5 else 1 for prob1 in row[1]]
                resultTempDF[resultColumns[2]] = [1 - prob1.numpy().tolist()[0] for prob1 in row[1]]
                resultTempDF[resultColumns[3]] = [prob1.numpy().tolist()[0] for prob1 in row[1]]
                resultTempDF[resultColumns[4]] = row[2]
                resultTempDF[resultColumns[5]] = row[3]
                resultTempDF[resultColumns[6]] = row[4]
                resultTempDF[resultColumns[7]] = row[5]
                resultTempDF[resultColumns[8]] = 'train'
                resultTempDF[resultColumns[9]] = row[6]
                resultTempDF[resultColumns[10]] = row[7]
                resultsDF = pd.concat([resultsDF, resultTempDF], axis=0)

            for row in resultsTs:
                resultTempDF = pd.DataFrame()
                resultTempDF[resultColumns[0]] = row[0]#.values
                resultTempDF[resultColumns[1]] = [0 if prob1.numpy().tolist()[0] < 0.5 else 1 for prob1 in row[1]]
                resultTempDF[resultColumns[2]] = [1 - prob1.numpy().tolist()[0] for prob1 in row[1]]
                resultTempDF[resultColumns[3]] = [prob1.numpy().tolist()[0] for prob1 in row[1]]
                resultTempDF[resultColumns[4]] = row[2]
                resultTempDF[resultColumns[5]] = row[3]
                resultTempDF[resultColumns[6]] = row[4]
                resultTempDF[resultColumns[7]] = row[5]
                resultTempDF[resultColumns[8]] = 'test'
                resultTempDF[resultColumns[9]] = row[6]
                resultTempDF[resultColumns[10]] = row[7]
                resultsDF = pd.concat([resultsDF, resultTempDF], axis=0)

            resultsDF.reset_index(inplace=True, drop=True)

            with open(save_path+'Results/speech_emo_e_'+str(experiment)+'.csv', 'a') as f1:
                resultsDF.to_csv(f1, mode='a', index=False, header=f1.tell()==0)


 exp:   0%|          | 0/1 [00:00<?, ?it/s]

 runs:   0%|          | 0/3 [00:00<?, ?it/s]

 tasks:   0%|          | 0/5 [00:00<?, ?it/s]

 ofolds:   0%|          | 0/4 [00:00<?, ?it/s]

 ofolds:   0%|          | 0/4 [00:00<?, ?it/s]

...

 ofolds:   0%|          | 0/4 [00:00<?, ?it/s]

 ofolds:   0%|          | 0/4 [00:00<?, ?it/s]

 ofolds:   0%|          | 0/4 [00:00<?, ?it/s]

 ofolds:   0%|          | 0/4 [00:00<?, ?it/s]

 ofolds:   0%|          | 0/4 [00:00<?, ?it/s]

# Results

In [ ]:
resPath = 'Text/Results/'
experiment = 14 # which experiment to read
resDF = pd.read_csv(path+resPath+'speech_emo_e_'+str(experiment)+'.csv', header=0) # collect predictions
resTestDF = resDF[resDF.source=='test'] # use the test data only

In [ ]:
runResDF = pd.DataFrame() # text 5 10 runs

for run in resTestDF.run.unique():
    tempDF = calSResults(resTestDF[resTestDF.run==run])
    tempDF['run'] = run
    runResDF = pd.concat([runResDF,tempDF], axis=0)

runResDF.groupby(['model','task']).agg(['mean','std']).round(2).reset_index(
).drop('run', axis=1, level=0).sort_values('task')

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


model task    acc         prec           rec            f1       
                  mean   std   mean    std   mean    std   mean    std
5  STMVCMV  0.0  59.53  2.93  63.43   8.36  16.22   8.64  24.73  11.39
6  STMVCWV  0.0  56.16  3.15  45.00  18.78  23.55  16.34  28.64  16.37
0     STMV  1.0  58.00  3.77  52.27  10.46  26.28  13.41  33.27  12.90
1     STMV  2.0  53.17  4.86  43.57  10.86  29.78  14.01  33.68  12.60
2     STMV  3.0  58.61  6.43  53.08   9.41  33.42  12.44  40.30  11.28
3     STMV  4.0  58.65  1.94  54.47   6.39  30.23  12.94  36.93  10.11
4     STMV  5.0  59.63  2.15  56.64  24.91  19.08  14.24  26.41  13.71

In [ ]:
runResDF = pd.DataFrame() # text 5

for run in resTestDF.run.unique():
    tempDF = calSResults(resTestDF[resTestDF.run==run])
    tempDF['run'] = run
    runResDF = pd.concat([runResDF,tempDF], axis=0)

runResDF.groupby(['model','task']).agg(['mean','std']).round(2).reset_index(
).drop('run', axis=1, level=0).sort_values('task')

model task    acc          prec           rec            f1       
                  mean    std   mean    std   mean    std   mean    std
5  STMVCMV  0.0  58.03   1.99  57.70   9.60  17.77   7.68  26.10   9.06
6  STMVCWV  0.0  54.50   3.36  42.40   9.47  21.47  15.14  26.70  16.39
0     STMV  1.0  57.00   6.24  48.03  10.37  31.00  18.65  36.27  17.81
1     STMV  2.0  54.17   5.28  47.23   6.32  37.77   3.87  41.63   2.41
2     STMV  3.0  57.17  11.76  51.27  17.21  33.37  16.75  39.50  16.89
3     STMV  4.0  59.57   2.03  60.33   9.03  28.03  19.33  34.53  14.04
4     STMV  5.0  59.27   1.67  57.30   4.59  16.67   5.69  25.60   7.25

In [ ]:
runResDF = pd.DataFrame() # text 8

for run in resTestDF.run.unique():
    tempDF = calSResults(resTestDF[resTestDF.run==run])
    tempDF['run'] = run
    runResDF = pd.concat([runResDF,tempDF], axis=0)

runResDF.groupby(['model','task']).agg(['mean','std']).round(2).reset_index(
).drop('run', axis=1, level=0).sort_values('task')

model task   acc      prec       rec        f1    
                 mean std  mean std  mean std  mean std
6  STMVCWV  0.0  62.5 NaN  71.4 NaN  22.2 NaN  33.9 NaN
5  STMVCMV  0.0  62.5 NaN  80.0 NaN  17.8 NaN  29.1 NaN
0     STMV  1.0  58.0 NaN  55.6 NaN  11.6 NaN  19.2 NaN
1     STMV  2.0  55.8 NaN  47.4 NaN  20.0 NaN  28.1 NaN
2     STMV  3.0  61.8 NaN  64.7 NaN  25.0 NaN  36.1 NaN
3     STMV  4.0  61.2 NaN  59.1 NaN  29.5 NaN  39.4 NaN
4     STMV  5.0  60.2 NaN  56.0 NaN  31.8 NaN  40.6 NaN